# YOLO Model Training

The YOLO model is the model that is used to predict a bounding box around worm's head.
In order for the model to make successful detections, the model must be trained on example images, with corresponding annotations (labels). 
The set of example images with their corresponding annotations is called a dataset.
In this notebook, the YOLO model is trained on a dataset obtained from RoboFlow.

RoboFlow is a website specifically designed for creating datasets for neural-network models, and YOLO among them.
We decided to use Reboflow because of the ease-of-use it provides in the creation of new datasets, and in the process of data annotation.

In [ ]:
import ultralytics
from ultralytics import YOLO
import numpy as np

from utils.gui_utils import UserPrompt
from utils.path_utils import join_paths

In [ ]:
################################ User Input ################################

# the folder which contains the dataset, or into which the dataset will be downloaded
# if None, the user will be prompted to select a folder
dataset_folder = "data/yolo/CElegan-Head-5"

# path to the YOLO training configuration file.
# if None, the user will be prompted to select a file
train_config_file = "yolo/yolo_train_config.yaml"

############################################################################

if dataset_folder is None:
    dataset_folder = UserPrompt.open_directory("Select Dataset Folder")

if train_config_file is None:
    train_config_file = UserPrompt.open_file("Select YOLO Training Configuration File", file_types=[("YAML files", "*.yaml")])

dataset_file = join_paths(dataset_folder, "data.yaml")

print(f"Dataset Folder: {dataset_folder}")
print(f"Training Config: {train_config_file}")

If Roboflow is used, copy the code produced by Roboflow to the cell below and run it to download the dataset.   
Otherwise, if the dataset is already on the system there is no need to do this step.

In [ ]:
from roboflow import Roboflow

################################ User Input ################################

rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("technion-institute-of-technology").project("celegan-head")
version = project.version(5)

############################################################################

dataset = version.download("yolov8", location=dataset_folder, overwrite=False)

### Load a base model

The loaded model can be completely untrained YOLO weights, or can be a pre-trained model, which was previously trained.  
It is recommended to use pre-trained models if possible, since their training process is usually faster.

In [ ]:
################################ User Input ################################

# can be one of the models stated at the ultraliytics documentation or a 
# path to a trained model on the disk
model_path = "yolov8s"  

############################################################################

if model_path is None:
    yolo_model_path = UserPrompt.open_file("Select YOLO model", ("pytorch file", "*.pt"))

model = YOLO(model=model_path)

**Train The Model**

In [ ]:
results = model.train(data=dataset_file, cfg=train_config_file)

**Export to faster format**

In [ ]:
model = YOLO(model=model_path)

out = model.export(
    format="engine",  # Use 'engine' for TensorRT (GPU), 'onnx' for ONNX (CPU/GPU).
    imgsz=384,
    half=False,  # Enables FP16 (half-precision) quantization, reducing model size and potentially speeding up inference on supported hardware.
    dynamic=False,  # Allows dynamic input sizes for ONNX and TensorRT exports, enhancing flexibility in handling varying image dimensions
    simplify=False,  # Simplifies the model graph for ONNX exports, potentially improving performance and compatibility
    verbose=False,
    batch=8,  # max supported batch size of the exported model
    workspace=8,  # max memory usage during the export process in GB
)

**Deploy Model to Roboflow**

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8", model_path=model_path)